# Running an analysis

The `solver` module is used to interact with OpenSees.

The most convenient approach is using a child of the `Analysis` class from those already defined in the `solver` module for the specific analysis that is needed. This will run the analysis and store the requested results of each analysis step.

Alternatively, a generic `Analysis` object can be defined (from the parent class), and its generic methods for model definition can be used together with OpenSees commands issued directly in the analysis script. If such an analysis is repeated a lot, a new `Analysis` child class can be defined in the `solver` module.

## Available analysis objects

First, we need to define a model.

In [ ]:
# This cell defines the model from notebook 2_Define_a_Model
# with distributed plasticity elements
import sys
sys.path.append('../src')
import model
import numpy as np

b = model.Model()
b.add_level('base', 0.00, 'fixed')
for i in range(3):
    b.add_level(str(i+1), 144.00*(i+1))
b.set_active_material('steel-bilinear-fy50')
b.add_sections_from_json(
    "../section_data/sections.json",
    'W',
    ["W24X94"])
b.set_active_section("W24X94")
b.set_active_levels("all_above_base")
b.assign_surface_load(1.00)
b.active_placement = 'centroid'
b.set_active_section("W24X94")
p1 = np.array((0.00, 0.00))
p2 = np.array((360., 0.00))
p3 = np.array((360., 360.))
p4 = np.array((0.00, 360.00))
for pt in [p1, p2, p3, p4]:
    b.add_column_at_point(
        pt,
        n_sub=10,
        model_as={'type': 'fiber',
                 'n_x': 10,
                 'n_y': 10,
                 'n_p': 5})

b.active_placement = 'top_center'
for pair in ((p1, p2), (p2, p3), (p3, p4), (p4, p1)):
    b.add_beam_at_points(
        pair[0], pair[1],
        n_sub=10,
        model_as={'type': 'fiber',
                 'n_x': 10,
                 'n_y': 10,
                 'n_p': 5})
b.select_perimeter_beams_all()
b.selection.add_UDL(np.array((0.00, 0.00, -20.00)))
import preprocess
preprocess.diaphragms(b)
preprocess.tributary_area_analysis(b)
preprocess.self_weight_and_mass(b)

### Linear static

In [ ]:
# Additional nodal loads can be applied if desired
for node in b.list_of_parent_nodes():
    node.load += np.array([0.00, 0.00, 0.00, 0.00, 0.00, 0.00])

In [ ]:
import solver

In [ ]:
# instantiate analysis object
linear_gravity_analysis = solver.LinearGravityAnalysis(b)

In [ ]:
linear_gravity_analysis.run()

#### Retrieving results

Analysis results are stored in dictionaries. The keys correspond to the unique identifyer of the element that the results correspond to (but it needs to be converted to a string), and the value contains the results.

In [ ]:
# Show all node displacement results
# DANGER: Don't do this for a large model.
# linear_gravity_analysis.node_displacements

In [ ]:
# Displacement of the parent node of the top story,
# in the Y direction.
analysis_step = 0
direction = 1
node_id = b.list_of_parent_nodes()[-1].uid
linear_gravity_analysis.node_displacements[
    str(node_id)][analysis_step][direction]

Note: Multiple analysis objects can be defined using the same model. The results will be sotored independently in the various analysis objects. We can take advantage of this in the case of linear static analyses. For instance, we can define one analysis object for each load case and then superimpose the results as desired.

#### Visualizing results

The following visualization methods work for all analysis methods. However, some require specifying the analysis step to visualize. Linear static analysis only has a single step, so this is not required.

In [ ]:
linear_gravity_analysis.deformed_shape(extrude_frames=False)
linear_gravity_analysis.deformed_shape(extrude_frames=True)

In [ ]:
linear_gravity_analysis.basic_forces()

Sadly, generating the plots is currently quite inefficient right now. There is potential for major performance enhancements in `postprocessing_3D.py`.

More visualization methods can be written, following the implementation steps of the existing ones.

### Modal

In [ ]:
modal_analysis = solver.ModalAnalysis(b, num_modes=3)
modal_analysis.run()

In [ ]:
print(modal_analysis.periods)

In [ ]:
modal_analysis.deformed_shape(step=0, scaling=0.00, extrude_frames=True)

### Pushover

In [ ]:
pushover_analysis = solver.PushoverAnalysis(b)
control_node = b.list_of_all_nodes()[-1]  # top floor
analysis_metadata = pushover_analysis.run(
    "y",
    np.array([10.]),
    control_node,
    1./2.)
n_plot_steps = analysis_metadata['successful steps']

In [ ]:
# plot the deformed shape for any of the steps
plot_metadata = pushover_analysis.deformed_shape(
    step=n_plot_steps-1, scaling=0.00, extrude_frames=True)
print(plot_metadata)

In [ ]:
# plot pushover curve
pushover_analysis.plot_pushover_curve("y", control_node)

In [ ]:
pushover_analysis.basic_forces(step=n_plot_steps-1)

### Time-history

In [ ]:
nlth = solver.NLTHAnalysis(b)

In [ ]:
nlth.plot_ground_motion('groundmotions/1xa.txt', 0.005, plotly=True)

In [ ]:
nlth.run(0.01,
         'groundmotions/1xa.txt',
         'groundmotions/1xa.txt',
         None, 0.005,
        damping={'type': 'rayleigh', 'ratio': 0.05, 'periods': [1.00, 0.30]},
        printing=False)

In [ ]:
node = b.list_of_parent_nodes()[-1]  # top floor
nlth.plot_node_displacement_history(node, 0, plotly=True)
nlth.plot_node_displacement_history(node, 1, plotly=True)

In [ ]:
nlth.deformed_shape(435, scaling=0.00, extrude_frames=True)